In [8]:
'''从零开始实现线性回归'''
%matplotlib inline
import random
import torch
#构造数据集
def synthetic_data(w,b,num):
    #生成均值为0，标准差为1的随机数，大小为num，列数为len(w)（即得到num组数据）
    x = torch.normal(0,1,(num,len(w)))
    #令y = wx+b
    y = torch.matmul(x,w) + b
    #加入噪音项
    y += torch.normal(0,0.01,y.shape)
    return x,y.reshape((-1,1))

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = synthetic_data(true_w,true_b,1000)
print('features',features[0],'labels',labels[0])

features tensor([-0.6877, -0.1976]) labels tensor([3.5012])


In [9]:
def data_iter(batch_size,features,labels):
    #得到features的个数
    num = len(features)
    #生成index
    indices = list(range(num))
    #随机打乱样本
    random.shuffle(indices)
    #取出从i到i+batch_size的数据
    for i in range(0,num,batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num)])
        #返回当前小批量的特征和标签
        yield features[batch_indices],labels[batch_indices]

batch_size = 10
for x,y in data_iter(batch_size,features,labels):
    print(x)
    print(y)
    break

tensor([[ 0.4464, -1.1667],
        [ 0.7391, -0.0027],
        [-1.3304,  1.3782],
        [-0.1947,  0.7827],
        [ 1.2469, -0.7217],
        [-0.4174, -0.5166],
        [-0.7747,  0.3368],
        [-2.0648, -1.8356],
        [-0.6704, -2.1659],
        [ 0.4317,  0.5862]])
tensor([[ 9.0499],
        [ 5.7068],
        [-3.1459],
        [ 1.1502],
        [ 9.1432],
        [ 5.1424],
        [ 1.4856],
        [ 6.3159],
        [10.2354],
        [ 3.0836]])


In [10]:
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

def linreg(x,w,b):
    '''线性回归模型'''
    return torch.matmul(x,w) + b

print(w)
print(b)

tensor([[ 4.2079e-05],
        [-1.0471e-02]], requires_grad=True)
tensor([0.], requires_grad=True)


In [11]:
def squared_loss(y_hat,y):
    '''均方损失'''
    return (y_hat - y.reshape(y_hat.shape))**2/2

In [12]:
def sgd(params,lr,batch_size):
    '''小批量随机梯度下降'''
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad/batch_size
            param.grad.zero_()

In [13]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for x,y in data_iter(batch_size,features,labels):
        l = loss(net(x,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch {epoch+1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.036104
epoch 2, loss 0.000132
epoch 3, loss 0.000049
